# **Signal To Spike Conversion** - Baseline Detection
This notebook presents the **baseline detection** step that needs to be conducted before doing the signal-to-spike conversion.

The signal-to-spike conversion requires the definition of the `up_threshold` and `down_threshold` values that will be used to translate the continuous signal into discrete events. Here, we follow the approach proposed by Indiveri's team to define these thresholds. Which is composed of the following steps:
1. Select a time window of the signal representative of the whole signal.
   - The source work used a 1s window.
   - We will test different window sizes.
2. Store the maximum signal amplitudes of consecutive non-overlapping time sub-windows of 50ms.
3. Take the mean of the lowest quartile as the baseline amplitude.
   - Check why we are using the lowest quartile.    

### Check WD (change if necessary) and file loading

In [1]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/hfo/signal_to_spike" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/hfo/signal_to_spike"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

/home/monkin/Desktop/feup/thesis
File Location:  /home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/signal_to_spike
New Working Directory:  /home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/signal_to_spike


## Let's first load the filtered SEEG signal
This notebook only converts 1 SEEG channel to spikes. Therefore, the `.npy` file must contain a single channel.

### Declare the `INPUT_FOLDER` and `RESULTS_FOLDER`

In [2]:
# CAREFUL WITH THIS FOLDER TO NOT OVERWRITE THE FILES
INPUT_FOLDER = "../subset/results/seeg_filtered_subset_90-119_segment500_200"
RESULTS_FOLDER = "results/custom_subset_90-119_segment500_200"

### Load the Ripple, Fast Ripple, and Annotated Events

In [3]:
import numpy as np
import math
from utils.io import preview_np_array

# Load the filtered seeg signal in the ripple band
ripple_seeg_file_name = f"{INPUT_FOLDER}/ripple_band.npy"
ripple_band_seeg = np.load(f"{ripple_seeg_file_name}")

# Remove the extra inner dimension
ripple_band_seeg = np.squeeze(ripple_band_seeg)

preview_np_array(ripple_band_seeg, "Ripple Band SEEG", edge_items=3)

Ripple Band SEEG Shape: (245760,).
Preview: [1.84829940e-04 1.37975809e-03 3.92964380e-03 ... 1.23691538e+00
 7.54000855e-01 1.73992494e-01]


In [4]:
# Load the filtered seeg signal in the fast ripple band
fr_seeg_file_name = f"{INPUT_FOLDER}/fr_band.npy"
fr_band_seeg = np.load(f"{fr_seeg_file_name}")

# Remove the extra inner dimension
fr_band_seeg = np.squeeze(fr_band_seeg)

preview_np_array(fr_band_seeg, "FR Band SEEG", edge_items=3)

FR Band SEEG Shape: (245760,).
Preview: [ 0.00095524  0.00310525 -0.00478473 ... -0.49596476 -0.2753618
  0.28661303]


In [5]:
# Load the annotated events (For Fast Ripples)
fr_markers_file_name = f"{INPUT_FOLDER}/markers_fr_band.npy"
fr_markers = np.load(f"{fr_markers_file_name}")

# Remove the extra inner dimension
fr_markers = np.squeeze(fr_markers)

preview_np_array(fr_markers, "fr_markers", edge_items=3)

fr_markers Shape: (199,).
Preview: [('Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Fast-Ripple',   3770.02, 0.) ... ('Fast-Ripple', 116096.  , 0.)
 ('Ripple+Fast-Ripple', 116769.  , 0.) ('Fast-Ripple', 119000.  , 0.)]


## Select a time window of the signal representative of the whole signal

In [ ]:
window_size = 10 * (10**3)  # 10 seconds or 10,000 ms

# Crop the signal to the first 10 seconds